# Classical Music Generation
This model notebook is from [Coursera](https://www.coursera.org/learn/nlp-sequence-models/notebook/cxJXc/jazz-improvisation-with-lstm).

In [ ]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
import tensorflow as tf
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, Bidirectional
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.backend import reshape
from keras import backend as K

### Helper function to create one-hot vectors

In [2]:
def one_hot(x):
    x = K.argmax(x)
    x = tf.one_hot(x, n_values) 
    x = RepeatVector(1)(x)
    return x

### Load data from preprocessing files

In [3]:
X, Y, n_values, indices_values = load_music_utils()

### Convert data to single precision FPs

In [4]:
X = X.astype(np.float32)
Y = Y.astype(np.float32)

### Print out information about the training data

In [5]:
print('shape of X:', X.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)

shape of X: (60, 30, 69)
number of training examples: 60
Tx (length of sequence): 30
total # of unique values: 69
Shape of Y: (30, 60, 69)


### Define parameters for the LSTM RNN

In [6]:
n_a = 64 
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state = True)
densor = Dense(n_values, activation='softmax')

In [7]:
def create_model(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: X[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs = [X, a0, c0], outputs = outputs)
    
    return model

In [8]:
model = create_model(Tx = 30 , n_a = n_a, n_values = n_values)

### Train the model

In [9]:
opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, decay=0.003)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

In [11]:
model.fit([X, a0, c0], list(Y), epochs=100)

Epoch 1/100
60/60 [==============================] - 11s 177ms/step - loss: 117.3599 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.0167 - dense_1_acc_1: 0.1667 - dense_1_acc_2: 0.2000 - dense_1_acc_3: 0.1167 - dense_1_acc_4: 0.1333 - dense_1_acc_5: 0.2000 - dense_1_acc_6: 0.1000 - dense_1_acc_7: 0.1000 - dense_1_acc_8: 0.1333 - dense_1_acc_9: 0.1333 - dense_1_acc_10: 0.1167 - dense_1_acc_11: 0.1167 - dense_1_acc_12: 0.1333 - dense_1_acc_13: 0.1000 - dense_1_acc_14: 0.0667 - dense_1_acc_15: 0.1833 - dense_1_acc_16: 0.1167 - dense_1_acc_17: 0.0833 - dense_1_acc_18: 0.1000 - dense_1_acc_19: 0.0833 - dense_1_acc_20: 0.0667 - dense_1_acc_21: 0.1000 - dense_1_acc_22: 0.1500 - dense_1_acc_23: 0.0833 - dense_1_acc_24: 0.1333 - dense_1_acc_25: 0.1500 - dense_1_acc_26: 0.1667 - dense_1_acc_27: 0.1000 - dense_1_acc_28: 0.1500 - dense_1_acc_29: 0.0333                                                                                 
Epoch 2/100
60/60 [==============================] - 0s 1ms/step - lo

60/60 [==============================] - 0s 2ms/step - loss: 4.2198 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1_acc_2: 0.9500 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.9667 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 20/100
60/60 [==============================] - 0s 1ms/step - loss: 4.1581 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1

60/60 [==============================] - 0s 1ms/step - loss: 3.8510 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1_acc_2: 0.9667 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.9667 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 38/100
60/60 [==============================] - 0s 2ms/step - loss: 3.8500 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1

60/60 [==============================] - 0s 2ms/step - loss: 3.7830 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.8833 - dense_1_acc_2: 0.9500 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.9667 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 56/100
60/60 [==============================] - 0s 2ms/step - loss: 3.7856 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.8833 - dense_1

60/60 [==============================] - 0s 2ms/step - loss: 3.7466 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1_acc_2: 0.9667 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.9667 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 74/100
60/60 [==============================] - 0s 2ms/step - loss: 3.7484 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1

60/60 [==============================] - 0s 2ms/step - loss: 3.7341 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1_acc_2: 0.9667 - dense_1_acc_3: 1.0000 - dense_1_acc_4: 0.9667 - dense_1_acc_5: 1.0000 - dense_1_acc_6: 1.0000 - dense_1_acc_7: 1.0000 - dense_1_acc_8: 1.0000 - dense_1_acc_9: 1.0000 - dense_1_acc_10: 1.0000 - dense_1_acc_11: 1.0000 - dense_1_acc_12: 1.0000 - dense_1_acc_13: 1.0000 - dense_1_acc_14: 1.0000 - dense_1_acc_15: 1.0000 - dense_1_acc_16: 1.0000 - dense_1_acc_17: 1.0000 - dense_1_acc_18: 1.0000 - dense_1_acc_19: 1.0000 - dense_1_acc_20: 1.0000 - dense_1_acc_21: 1.0000 - dense_1_acc_22: 1.0000 - dense_1_acc_23: 1.0000 - dense_1_acc_24: 1.0000 - dense_1_acc_25: 1.0000 - dense_1_acc_26: 1.0000 - dense_1_acc_27: 1.0000 - dense_1_acc_28: 1.0000 - dense_1_acc_29: 0.0000e+00
Epoch 92/100
60/60 [==============================] - 0s 2ms/step - loss: 3.7317 - dense_1_loss: 0.0000e+00 - dense_1_acc: 0.1167 - dense_1_acc_1: 0.9000 - dense_1

### Generate Music from trained LSTM and Dense layers

In [12]:
def music_inference_model(LSTM_cell, densor, n_values = n_values, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x = Lambda(one_hot)(out)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs = [x0, a0, c0], outputs = outputs)
    
    return inference_model

In [13]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = n_values, n_a = n_a, Ty = 50)

### Generate Music based on the inference model
Here we use a tempo of 75 beats / min, and hand-picked chords to generate music.

In [26]:
def generate_music(inference_model, corpus = corpus, abstract_grammars = abstract_grammars, tones = tones, tones_indices = tones_indices, indices_tones = indices_tones, T_y = 10, max_tries = 1000, diversity = 0.5):
    """
    Generates music using a model trained to learn musical patterns of Haydn's Sonata No.48 Finale. Creates an audio stream
    to save the music and play it.
    
    Arguments:
    model -- Keras model Instance, output of djmodel()
    corpus -- musical corpus, list of tones as strings (ex: 'C,0.333,<P1,d-5>')
    abstract_grammars -- list of grammars, on element can be: 'S,0.250,<m2,P-4> C,0.250,<P4,m-2> A,0.250,<P4,m-2>'
    tones -- set of unique tones, ex: 'A,0.250,<M2,d-4>' is one element of the set.
    tones_indices -- a python dictionary mapping unique tone (ex: A,0.250,< m2,P-4 >) into their corresponding indices (0-77)
    indices_tones -- a python dictionary mapping indices (0-77) into their corresponding unique tone (ex: A,0.250,< m2,P-4 >)
    Tx -- integer, number of time-steps used at training time
    temperature -- scalar value, defines how conservative/creative the model is when generating music
    
    Returns:
    predicted_tones -- python list containing predicted tones
    """
    
    # set up audio stream
    out_stream = stream.Stream()
    
    # Initialize chord variables
    curr_offset = 0.0                                     # variable used to write sounds to the Stream.
    num_chords = int(len(chords) / 3)                     # number of different set of chords

    manual_selection = True
    if manual_selection:
        chord_list = [3, 5, 6]
        
        # Below are the bad chords
        #chord_list = [1, 2, 4, 7, 8]
    else:
        chord_list = range(1, num_chords)


    print("Predicting new values for different set of chords.")
    # Loop over the selected chords. At each iteration generate a sequence of tones
    # and use the current chords to convert it into actual sounds 
    for i in chord_list:
        
        # Retrieve current chord from stream
        curr_chords = stream.Voice()
        
        # Loop over the chords of the current set of chords
        for j in chords[i]:
            # Add chord to the current chords with the adequate offset, no need to understand this
            curr_chords.insert((j.offset % 4), j)
        
        # Generate a sequence of tones using the model
        _, indices = predict_and_sample(inference_model)
        indices = list(indices.squeeze())
        pred = [indices_tones[p] for p in indices]
        
        predicted_tones = 'C,0.25 '
        for k in range(len(pred) - 1):
            predicted_tones += pred[k] + ' ' 
        
        predicted_tones +=  pred[-1]
                
        #### POST PROCESSING OF THE PREDICTED TONES ####
        # We will consider "A" and "X" as "C" tones. It is a common choice.
        predicted_tones = predicted_tones.replace(' A',' C').replace(' X',' C')

        # Pruning #1: smoothing measure
        predicted_tones = prune_grammar(predicted_tones)
        
        # Use predicted tones and current chords to generate sounds
        sounds = unparse_grammar(predicted_tones, curr_chords)

        # Pruning #2: removing repeated and too close together sounds
        sounds = prune_notes(sounds)

        # Quality assurance: clean up sounds
        sounds = clean_up_notes(sounds)

        # Print number of tones/notes in sounds
        print('Generated %s sounds using the predicted values for the set of chords ("%s") and after pruning' % (len([k for k in sounds if isinstance(k, note.Note)]), i))
        
        # Insert sounds into the output stream
        for m in sounds:
            out_stream.insert(curr_offset + m.offset, m)
        for mc in curr_chords:
            out_stream.insert(curr_offset + mc.offset, mc)

        curr_offset += 4.0
        
    # Initialize tempo of the output stream with 75 bit per minute
    out_stream.insert(0.0, tempo.MetronomeMark(number=75))

    # Save audio stream to fine
    mf = midi.translate.streamToMidiFile(out_stream)
    mf.open("output/my_music.midi", 'wb')
    mf.write()
    print("Your generated music is saved in output/my_music.midi")
    mf.close()
    
    # Play the final stream through output (see 'play' lambda function above)
    # play = lambda x: midi.realtime.StreamPlayer(x).play()
    # play(out_stream)
    
    return out_stream

### Generate music and output the midi file

In [27]:
out_stream = generate_music(inference_model)

Predicting new values for different set of chords.
Generated 47 sounds using the predicted values for the set of chords ("3") and after pruning
Generated 47 sounds using the predicted values for the set of chords ("5") and after pruning
Generated 45 sounds using the predicted values for the set of chords ("6") and after pruning
Your generated music is saved in output/my_music.midi
